In [4]:
from pathlib import Path
import os, shutil

# ===== 사용자 경로 설정 =====
SRC = Path("/home/dw/ws_job_msislab/Golf_Project/data/for_study/20250909_merge_data_split")
DST_BASE = SRC.parent / "split_by_zone_vertical_only_01"  # 결과 저장 폴더명
DST_BASE.mkdir(exist_ok=True)

IMG_EXTS = {".jpg", ".jpeg", ".png"}

# 구역 폴더: 위→아래 1~4 (각각 images/, labels/ 구성)
zone_dirs = {
    i: {
        "root":   DST_BASE / f"zone_{i}",
        "images": DST_BASE / f"zone_{i}" / "images",
        "labels": DST_BASE / f"zone_{i}" / "labels",
    }
    for i in range(1, 5)
}
for z in zone_dirs.values():
    z["root"].mkdir(parents=True, exist_ok=True)
    z["images"].mkdir(parents=True, exist_ok=True)
    z["labels"].mkdir(parents=True, exist_ok=True)

def get_zone_vertical(y_center: float) -> int:
    """
    y_center: YOLO 0~1 비율(위=0, 아래=1)
    세로 4등분:
      [0.00,0.25) -> 1 (맨 위)
      [0.25,0.50) -> 2
      [0.50,0.75) -> 3
      [0.75,1.00] -> 4 (맨 아래)
    """
    if y_center < 0.25:
        return 1
    elif y_center < 0.5:
        return 2
    elif y_center < 0.75:
        return 3
    else:
        return 4

def link_or_copy(src: Path, dst: Path):
    """가능하면 하드링크, 실패 시 복사 (덮어쓰기 일관성)"""
    try:
        if dst.exists():
            dst.unlink()
        os.link(src, dst)
    except Exception:
        shutil.copy2(src, dst)

# ===== 통계 =====
scanned = 0
eligible_images = 0
skipped_no_txt = 0
skipped_empty_or_invalid = 0
skipped_contains_other = 0
skipped_no_01 = 0
copied_counts = {1: 0, 2: 0, 3: 0, 4: 0}   # 각 구역에 복사된 '이미지 수' (중복 포함)

for img_path in sorted(SRC.iterdir()):
    if img_path.suffix.lower() not in IMG_EXTS:
        continue
    scanned += 1

    txt_path = img_path.with_suffix(".txt")
    if not txt_path.exists():
        skipped_no_txt += 1
        continue

    try:
        with open(txt_path, "r") as f:
            lines = [ln.strip() for ln in f if ln.strip()]
    except Exception:
        skipped_empty_or_invalid += 1
        continue

    if not lines:
        skipped_empty_or_invalid += 1
        continue

    classes = []
    ys_01 = []  # cls 0/1의 y_center 리스트
    valid = True
    for ln in lines:
        parts = ln.split()
        if len(parts) < 5:
            valid = False
            break
        try:
            cls = int(parts[0])
            y_center = float(parts[2])
        except ValueError:
            valid = False
            break

        classes.append(cls)
        if cls in (0, 1):
            ys_01.append(y_center)

    if not valid:
        skipped_empty_or_invalid += 1
        continue

    # 0/1 외 클래스가 하나라도 있으면 제외
    if any(c not in (0, 1) for c in classes):
        skipped_contains_other += 1
        continue

    # 0/1 객체가 최소 1개 있어야 대상
    if not ys_01:
        skipped_no_01 += 1
        continue

    eligible_images += 1

    # === 모든 걸치는 구역에 '중복 저장' (images/, labels/)
    zones = sorted(set(get_zone_vertical(y) for y in ys_01))
    for z in zones:
        img_dst = zone_dirs[z]["images"] / img_path.name
        lbl_dst = zone_dirs[z]["labels"] / txt_path.name
        link_or_copy(img_path, img_dst)
        link_or_copy(txt_path, lbl_dst)
        copied_counts[z] += 1

# ===== 결과 출력 =====
print("=== 완료 (중복 포함 + images/labels 구조) ===")
print(f"[스캔한 이미지]         : {scanned}")
print(f"[대상(0/1만 포함)]      : {eligible_images}")
print(f"[제외: txt 없음]        : {skipped_no_txt}")
print(f"[제외: 비어있음/오류]   : {skipped_empty_or_invalid}")
print(f"[제외: 0/1 외 클래스]   : {skipped_contains_other}")
print(f"[제외: 0/1이 없음]      : {skipped_no_01}")
for i in range(1, 5):
    print(f"Zone {i} (위→아래): {copied_counts[i]}")
print(f"[결과 경로] {DST_BASE}")


=== 완료 (중복 포함 + images/labels 구조) ===
[스캔한 이미지]         : 3829
[대상(0/1만 포함)]      : 2552
[제외: txt 없음]        : 540
[제외: 비어있음/오류]   : 0
[제외: 0/1 외 클래스]   : 737
[제외: 0/1이 없음]      : 0
Zone 1 (위→아래): 1598
Zone 2 (위→아래): 1520
Zone 3 (위→아래): 1024
Zone 4 (위→아래): 624
[결과 경로] /home/dw/ws_job_msislab/Golf_Project/data/for_study/split_by_zone_vertical_only_01


In [7]:
from pathlib import Path
import os, shutil

# ===== 사용자 경로 설정 =====
SRC = Path("/home/dw/ws_job_msislab/Golf_Project/data/for_study/20250909_merge_data_split")
DST_BASE = SRC.parent / "split_by_zone_vertical_only_01_exclusive"  # 결과 저장 폴더명
DST_BASE.mkdir(exist_ok=True)

IMG_EXTS = {".jpg", ".jpeg", ".png"}

# 구역 폴더: 위→아래 1~4  (각각 images/, labels/ 하위 폴더 생성)
zone_dirs = {
    i: {
        "root": DST_BASE / f"zone_{i}",
        "images": DST_BASE / f"zone_{i}" / "images",
        "labels": DST_BASE / f"zone_{i}" / "labels",
    }
    for i in range(1, 5)
}
for z in zone_dirs.values():
    z["root"].mkdir(parents=True, exist_ok=True)
    z["images"].mkdir(parents=True, exist_ok=True)
    z["labels"].mkdir(parents=True, exist_ok=True)

def get_zone_vertical(y_center: float) -> int:
    """
    y_center: YOLO 0~1 비율(위=0, 아래=1)
    세로 4등분:
      [0.00,0.25) -> 1
      [0.25,0.50) -> 2
      [0.50,0.75) -> 3
      [0.75,1.00] -> 4
    """
    if y_center < 0.25:
        return 1
    elif y_center < 0.5:
        return 2
    elif y_center < 0.75:
        return 3
    else:
        return 4

def link_or_copy(src: Path, dst: Path):
    """가능하면 하드링크, 실패 시 복사"""
    try:
        if dst.exists():
            dst.unlink()  # 덮어쓰기 일관성
        os.link(src, dst)
    except Exception:
        shutil.copy2(src, dst)

# ===== 통계 =====
scanned = 0
eligible = 0             # 조건 충족(단일 구역)에 의해 복사된 이미지 수
skipped_no_txt = 0
skipped_empty_or_invalid = 0
skipped_contains_other = 0
skipped_no_01 = 0
skipped_multi_zone = 0   # 0/1이 여러 구역에 걸쳐 제외된 수
zone_counts = {1: 0, 2: 0, 3: 0, 4: 0}

for img_path in sorted(SRC.iterdir()):
    if img_path.suffix.lower() not in IMG_EXTS:
        continue
    scanned += 1

    txt_path = img_path.with_suffix(".txt")
    if not txt_path.exists():
        skipped_no_txt += 1
        continue

    try:
        with open(txt_path, "r") as f:
            lines = [ln.strip() for ln in f if ln.strip()]
    except Exception:
        skipped_empty_or_invalid += 1
        continue

    if not lines:
        skipped_empty_or_invalid += 1
        continue

    classes = []
    ys_01 = []
    valid = True

    for ln in lines:
        parts = ln.split()
        if len(parts) < 5:
            valid = False
            break
        try:
            cls = int(parts[0])
            y_center = float(parts[2])
        except ValueError:
            valid = False
            break

        classes.append(cls)
        if cls in (0, 1):
            ys_01.append(y_center)

    if not valid:
        skipped_empty_or_invalid += 1
        continue

    # 0/1 외 클래스 있으면 제외
    if any(c not in (0, 1) for c in classes):
        skipped_contains_other += 1
        continue

    # 0/1이 아예 없으면 제외
    if not ys_01:
        skipped_no_01 += 1
        continue

    # 모든 0/1 객체가 같은 단일 세로 구역에만 있어야 통과
    zones = set(get_zone_vertical(y) for y in ys_01)
    if len(zones) != 1:
        skipped_multi_zone += 1
        continue

    zone = zones.pop()
    # zone_i/images/ 와 zone_i/labels/로 분리 저장
    img_dst = zone_dirs[zone]["images"] / img_path.name
    lbl_dst = zone_dirs[zone]["labels"] / txt_path.name
    link_or_copy(img_path, img_dst)
    link_or_copy(txt_path, lbl_dst)

    zone_counts[zone] += 1
    eligible += 1

# ===== 결과 출력 =====
print("=== 완료 (Exclusive: 단일 구역만 포함 + images/labels 구조) ===")
print(f"[스캔한 이미지]         : {scanned}")
print(f"[복사된 이미지(적합)]   : {eligible}")
print(f"[제외: txt 없음]        : {skipped_no_txt}")
print(f"[제외: 비어있음/오류]   : {skipped_empty_or_invalid}")
print(f"[제외: 0/1 외 클래스]   : {skipped_contains_other}")
print(f"[제외: 0/1 없음]        : {skipped_no_01}")
print(f"[제외: 다중 구역]       : {skipped_multi_zone}")
for i in range(1, 5):
    print(f"Zone {i} (위→아래): {zone_counts[i]}")
print(f"[결과 경로] {DST_BASE}")


=== 완료 (Exclusive: 단일 구역만 포함 + images/labels 구조) ===
[스캔한 이미지]         : 3829
[복사된 이미지(적합)]   : 1046
[제외: txt 없음]        : 540
[제외: 비어있음/오류]   : 0
[제외: 0/1 외 클래스]   : 737
[제외: 0/1 없음]        : 0
[제외: 다중 구역]       : 1506
Zone 1 (위→아래): 383
Zone 2 (위→아래): 347
Zone 3 (위→아래): 232
Zone 4 (위→아래): 84
[결과 경로] /home/dw/ws_job_msislab/Golf_Project/data/for_study/split_by_zone_vertical_only_01_exclusive


In [9]:
# ===========================================
# YOLOv8 mAP Evaluation for a Zone Folder
#   - zone_1~4 중 원하는 폴더만 바꾸면 됨
#   - 모델의 8개 전체 클래스 이름으로 결과 출력
# ===========================================
from ultralytics import YOLO
from pathlib import Path
import yaml

# ===== 1) 설정 =====
# 평가할 존 폴더 (images/, labels/ 구조가 이미 갖춰져 있어야 함)
ZONE_ROOT = Path("/home/dw/ws_job_msislab/Golf_Project/data/for_study/"
                 "split_by_zone_vertical_only_01_exclusive/zone_1")

WEIGHT = Path("/home/dw/ws_job_msislab/Golf_Project/"
              "runs_yolo/20250909_merge_data_yolov8s/weights/best.pt")

# 모델이 학습한 전체 클래스 이름
ALL_CLASSES = [
    "Divot", "Fixed_Divot", "Diseased_Grass", "Pole",
    "Obstacle", "Sprinkler", "Drain", "Golf ball"
]

# ===== 2) 경로 점검 =====
IMAGES = ZONE_ROOT / "images"
LABELS = ZONE_ROOT / "labels"
assert WEIGHT.exists(), f"가중치 없음: {WEIGHT}"
assert IMAGES.exists(), f"이미지 폴더 없음: {IMAGES}"
assert LABELS.exists(), f"라벨 폴더 없음: {LABELS}"

# ===== 3) data.yaml 임시 생성 (train/val 동일 폴더 → 학습 안 함) =====
ZONE_NAME = ZONE_ROOT.name  # 예: "zone_3"
DATA_YAML = ZONE_ROOT / f"_{ZONE_NAME}_val.yaml"

data_cfg = {
    "train": str(IMAGES),   # 형식상 필요. 실제 학습 안 함
    "val":   str(IMAGES),   # 검증에 사용
    "nc": len(ALL_CLASSES),
    "names": ALL_CLASSES
}
with open(DATA_YAML, "w") as f:
    yaml.safe_dump(data_cfg, f, sort_keys=False)
print(f"[INFO] data.yaml 생성: {DATA_YAML}")

# ===== 4) 검증 실행 (배치 크기 자동 재시도) =====
model = YOLO(str(WEIGHT))
batch_candidates = [32, 16, 8, 4, 2, 1]
results = None
last_err = None

for bs in batch_candidates:
    try:
        print(f"[TRY] batch={bs}")
        results = model.val(
            data=str(DATA_YAML),
            imgsz=1024,     # 메모리 부족 시 896/640으로 줄이기
            batch=bs,
            device=0,       # GPU 없으면 'cpu'
            conf=0.001,
            iou=0.6,
            workers=4,
            verbose=True
        )
        print(f"[OK] batch={bs}")
        break
    except RuntimeError as e:      # 주로 CUDA OOM
        print(f"[WARN] batch={bs} 실패: {e}")
        last_err = e
    except Exception as e:
        print(f"[WARN] batch={bs} 기타 오류: {e}")
        last_err = e

if results is None:
    raise SystemExit(f"모든 배치 시도가 실패했습니다. 마지막 오류: {last_err}")

# ===== 5) 지표 출력 =====
box = results.box
print(f"\n==== {ZONE_NAME} Evaluation ====")
print(f"Precision      : {box.mp:.4f}")
print(f"Recall         : {box.mr:.4f}")
print(f"mAP@0.50       : {box.map50:.4f}")
print(f"mAP@0.50:0.95  : {box.map:.4f}")

maps = getattr(box, "maps", None)
if maps is not None:
    n = min(len(maps), len(ALL_CLASSES))
    if len(maps) != len(ALL_CLASSES):
        print(f"[NOTE] len(maps)={len(maps)} vs len(names)={len(ALL_CLASSES)} → 공통 길이 {n}까지만 출력")
    for cid in range(n):
        v = maps[cid]
        v_str = f"{v:.4f}" if isinstance(v, (float, int)) else str(v)
        print(f"Class {cid} ({ALL_CLASSES[cid]}): mAP@0.5:.95 = {v_str}")

print(f"\n[RESULTS SAVED] {results.save_dir}")






[INFO] data.yaml 생성: /home/dw/ws_job_msislab/Golf_Project/data/for_study/split_by_zone_vertical_only_01_exclusive/zone_1/_zone_1_val.yaml
[TRY] batch=32
Ultralytics 8.3.175 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 7932MiB)
Model summary (fused): 72 layers, 11,128,680 parameters, 0 gradients, 28.5 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 3487.0±2418.3 MB/s, size: 2152.4 KB)


val: Scanning /home/dw/ws_job_msislab/Golf_Project/data/for_study/split_by_zone_vertical_only_01_exclusive/zone_1/labels... 383 images, 0 backgrounds, 0 corrupt: 100%|██████████| 383/383 [00:00<00:00, 3076.33it/s]

val: New cache created: /home/dw/ws_job_msislab/Golf_Project/data/for_study/split_by_zone_vertical_only_01_exclusive/zone_1/labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 12/12 [00:06<00:00,  1.92it/s]


                   all        383        691       0.94      0.843      0.934      0.825
                 Divot        257        370       0.93      0.786      0.907      0.791
           Fixed_Divot        192        321      0.951      0.899      0.961       0.86
Speed: 1.4ms preprocess, 11.4ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs/detect/val57
[OK] batch=32

==== zone_1 Evaluation ====
Precision      : 0.9401
Recall         : 0.8427
mAP@0.50       : 0.9342
mAP@0.50:0.95  : 0.8254
Class 0 (Divot): mAP@0.5:.95 = 0.7908
Class 1 (Fixed_Divot): mAP@0.5:.95 = 0.8600
Class 2 (Diseased_Grass): mAP@0.5:.95 = 0.8254
Class 3 (Pole): mAP@0.5:.95 = 0.8254
Class 4 (Obstacle): mAP@0.5:.95 = 0.8254
Class 5 (Sprinkler): mAP@0.5:.95 = 0.8254
Class 6 (Drain): mAP@0.5:.95 = 0.8254
Class 7 (Golf ball): mAP@0.5:.95 = 0.8254

[RESULTS SAVED] runs/detect/val57


In [6]:
# ===========================================
# zone_4 이미지 추론 + confidence 기반 결과 저장
# ===========================================
from ultralytics import YOLO
from pathlib import Path

# ===== 1) 경로 및 설정 =====
ZONE4_IMAGES = Path("/home/dw/ws_job_msislab/Golf_Project/data/for_study/"
                    "split_by_zone_vertical_only_01/zone_4/images")
WEIGHT = Path("/home/dw/ws_job_msislab/Golf_Project/"
              "runs_yolo/20250909_merge_data_yolov8s/weights/best.pt")

# confidence threshold (필요 시 조절)
CONF_THRES = 0.25   # 0.25 이상일 때만 박스 표시
IMG_SIZE   = 1024   # 추론 해상도
SAVE_DIR   = Path("/home/dw/ws_job_msislab/Golf_Project/runs_yolo/zone4_infer_results")

assert WEIGHT.exists(), f"가중치 없음: {WEIGHT}"
assert ZONE4_IMAGES.exists(), f"이미지 폴더 없음: {ZONE4_IMAGES}"
SAVE_DIR.mkdir(parents=True, exist_ok=True)

# ===== 2) 모델 로드 =====
model = YOLO(str(WEIGHT))

# ===== 3) 추론 실행 =====
# source 에 폴더 경로를 주면 해당 폴더 내 모든 이미지 추론
pred_results = model.predict(
    source=str(ZONE4_IMAGES),
    imgsz=IMG_SIZE,
    conf=CONF_THRES,
    save=True,              # 결과 이미지 저장
    project=str(SAVE_DIR),  # 저장할 상위 폴더
    name="pred_conf",       # 하위 폴더 이름
    device=0,               # GPU 없으면 'cpu'
    save_txt=False,         # 라벨 텍스트 저장 원하면 True
    save_conf=True          # txt에 confidence도 저장
)

print(f"[DONE] 결과 이미지와 라벨이 다음 위치에 저장됨:\n{SAVE_DIR / 'pred_conf'}")

# ===== 4) 개별 이미지별 confidence 로그 출력 (원한다면)
for r in pred_results:
    img_name = Path(r.path).name
    # r.boxes.conf: tensor of confidence scores
    if r.boxes is not None:
        confs = [float(c) for c in r.boxes.conf]
        print(f"{img_name} -> {confs}")
    else:
        print(f"{img_name} -> No detections")


image 1/666 /home/dw/ws_job_msislab/Golf_Project/data/for_study/split_by_zone_vertical_only_01/zone_4/images/20250721_053640_0006_000066_f000650_1920x1536_173.jpg: 832x1024 2 Fixed_Divots, 7.2ms
image 2/666 /home/dw/ws_job_msislab/Golf_Project/data/for_study/split_by_zone_vertical_only_01/zone_4/images/20250721_053640_0006_000068_f000670_1920x1536_175.jpg: 832x1024 2 Fixed_Divots, 7.3ms
image 3/666 /home/dw/ws_job_msislab/Golf_Project/data/for_study/split_by_zone_vertical_only_01/zone_4/images/20250721_053640_0006_000079_f000780_1920x1536_208.jpg: 832x1024 2 Divots, 1 Fixed_Divot, 7.8ms
image 4/666 /home/dw/ws_job_msislab/Golf_Project/data/for_study/split_by_zone_vertical_only_01/zone_4/images/20250721_053640_0006_000079_f000780_1920x1536_209.jpg: 832x1024 2 Divots, 1 Fixed_Divot, 6.9ms
image 5/666 /home/dw/ws_job_msislab/Golf_Project/data/for_study/split_by_zone_vertical_only_01/zone_4/images/20250721_053640_0006_000079_f000780_1920x1536_210.jpg: 832x1024 2 Divots, 1 Fixed_Divot, 6.6